# Ch 4: Case Study - Interface Design

This chapter presents a case study that demonstrates a process for designing functions that work together.  We will use *Turtle Graphics*, which is a way to create programmatic drawings.  Since *Turtle Graphics* is not part of Julia, we are going to use a Python module called `turtle`.  

A module is a file that contains a collection of related functions.  Packages contain one or more such modules to provide additional functionalities to our program.  Packages for Julia can be found in https://juliaobserver.com

Packages can be installed in the REPL by entering the Pkg mode by pressing the key ] and then entering 

`add "package name"`



In [1]:
using Pkg            
# this module allows us to manage modules and packages in Julia

Pkg.add("PyCall")    
# to call Python functions within Julia, we import a package called PyCall

using PyCall

  Updating registry at `C:\Users\eddyj\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `C:\Users\eddyj\.julia\environments\v1.2\Project.toml`
 [no changes]
  Updating `C:\Users\eddyj\.julia\environments\v1.2\Manifest.toml`
 [no changes]


┌ Info: Precompiling PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0]
└ @ Base loading.jl:1242


In [2]:
jturtle = pyimport("turtle")

PyObject <module 'turtle' from 'C:\\Python37\\lib\\turtle.py'>

In [3]:
my_t = jturtle.Turtle()

PyObject <turtle.Turtle object at 0x00000000219F4188>

Let's write a code to have a turtle draw a square.

In [ ]:
my_t.reset()
my_t.speed(1)

for i in 1:4
    my_t.forward(100)
    my_t.left(90)
end

### Encapsulation

Wrapping a piece of code up in a function is called encapsulation.  This allows us to reuse the code, which is more concise than copying and pasting the body again and again.

First, let's define a function that takes an instance of Turtle as an argument and draws a square.

In [6]:
function square(t)
    for i in 1:4
        t.forward(100)
        t.left(90)
    end
end

square (generic function with 1 method)

In [7]:
my_t.reset()
my_t.speed(1)
square(my_t)

### Generalization

Next, we add an argument (arguments) to a function.  This is called generalization because it makes the function more general.

In [8]:
function square(t, len)
    for i in 1:4
        t.forward(len)
        t.left(90)
    end
end

square (generic function with 2 methods)

In [9]:
my_t.reset()
my_t.speed(1)
square(my_t,50)

We can do another generalization where we define a function `polygon` that can draw a polygon with a given number of sides.

In [10]:
function polygon(t, n, len)
    angle = 360 / n
    for i in 1:n
        t.forward(len)
        t.left(angle)
    end
end

polygon (generic function with 1 method)

In [11]:
my_t.reset()
my_t.speed(1)
polygon(my_t,6,50)

### Interface Design

The next step is to write a function circle that takes a radius as one of the arguments.  One simple solution is to use `polygon` with a big value of $n$, like 50.

In [12]:
function circle(t, r)
    
    circumference = 2 * π * r
    n = 50
    len = circumference / n
    
    polygon(t, n, len)
    
end

circle (generic function with 1 method)

In [13]:
my_t.reset()
my_t.speed(1)
circle(my_t,50)

One limitation of this solution is that $n$ is an arbitrary constant.  If the radius is small, we may end up wasting time drawing very small segments.  In contrast, if the radius is very big, then the line segments may be too long.

We might choose to add $n$ as an additional argument of `polygon`.  But this would be inconvenient and the interface would be less clean.  $n$ simply pertains to the details of how the circle will be rendered.

The interface of a function is a summary of how it is used:  

- What are the arguments?
- What does the function do?
- What is the return valye?


Here is one potential solution where an appropriate value of $n$ is chosen inside the function depending on the circumference.

In [14]:
function circle(t, r)
    circumference = 2 * π * r
    
    n = trunc(circumference / 3) + 3  
    # the legnth of segment is approximately 3.  
    # Adding 3 guarantees that there will be at least 3 sides
    
    len = circumference / n
    polygon(t, n, len)
end

circle (generic function with 1 method)

In [15]:
my_t.reset()
my_t.speed(1)
circle(my_t,50)

### Refactoring

What if we now want a function to draw an arc?  For `circle`, we were able to re-use `polygon`.  But an arc is not a polygon.

One approach is to start with a copy of `polygon` and modify it in to write a function called `arc`.

In [16]:
function arc(t, r, angle)
    
    arc_len = 2 * π * r * angle / 360
    
    n = trunc(arc_len / 3) + 1
    
    step_len = arc_len / n
    step_angle = angle / n
    
    for i in 1:n
        t.forward(step_len)
        t.left(step_angle)
    
    end
    
end

arc (generic function with 1 method)

The second half of the function looks like `polygon` but we can't re-use it without changing its interface (that is, adding a third argument *angle*).  Also, `polygon` would not be a right name any more.

Let's now take a different route and define a more general function `polyline`.  We then rewrite `polygon`, `circle`, and `arc` using this new function.  

This process of rearranging a program to improve interfaces and facilitates code re-use is called **refactoring**.

In [17]:
"""
polyline(t, n, len, angle)
This is a docstring.
Draws n line segments with the given length and
angle (in degrees) between them. t is a turtle.
"""
function polyline(t, n, len, angle)
    for i in 1:n
        t.forward(len)
        t.left(angle)
    end
end

polyline

In [18]:
function polygon(t, n, len)
    angle = 360 / n
    polyline(t, n, len, angle)
end

function arc(t, r, angle)
    arc_len = 2 * π * r * angle / 360
    n = trunc(arc_len / 3) + 1
    step_len = arc_len / n
    step_angle = angle / n
    polyline(t, n, step_len, step_angle)
end

function circle(t, r)
    arc(t, r, 360)
end

circle (generic function with 1 method)

In [19]:
my_t.reset()
my_t.speed(1)
polygon(my_t, 4, 100)
circle(my_t,50)
arc(my_t,100,150)

### A Development Plan

A development plan is a process for writing programs. The process we used in this case study is “encapsulation and generalization”. The steps of this process are:

1. Start by writing a small program with no function definitions.
2. Once you get the program working, identify a coherent piece of it, encapsulate that piece in a function and give it a name.
3. Generalize the function definition by adding appropriate parameters.
4. Repeat steps 1–3 until you have a set of working functions. Copy and paste working code to avoid retyping (and redebugging).
5. Look for opportunities to improve the program by refactoring. For example, if you have similar code in several places, consider factoring it into an appropriately general function.

This process has some drawbacks—we will see alternatives later—but it can be useful if you don’t know ahead of time how to divide the program into functions. This approach lets you design as you go along.

### Some Comments on Debugging

An interface is like a contract between a function and a caller. The caller agrees to provide certain parameters and the function agrees to do certain work.

For example, `polyline` requires four arguments: $t$ has to be a turtle; $n$ has to be an integer; $len$ should be a positive number; and $angle$ has to be a number, which is understood to be in degrees.

These requirements are called preconditions because they are supposed to be true before the function starts executing.

Conversely, conditions at the end of the function are postconditions. Postconditions include the intended effect of the function (like drawing line segments) and any side effects (like moving the turtle or making other changes).

Preconditions are the responsibility of the caller. If the caller violates a (properly documented!) precondition and the function doesn’t work correctly, the bug is in the caller, not the function.

If the preconditions are satisfied and the postconditions are not, the bug is in the function. If your pre- and postconditions are clear, they can help with debugging.


### Exercise 4-8

The version of `arc` above is not very accurate because the linear approximation of the circle is always outside the true circle. As a result, the turtle ends up a few pixels away from the correct destination. The  solution below shows a way to reduce this error. Read the code and see if it makes sense to you. If you draw a diagram, you might see how it works.

In [20]:
"""
arc(t, r, angle)
Draws an arc with the given radius and angle:
t: turtle
r: radius
angle: angle subtended by the arc, in degrees
"""
function arc(t, r, angle)

    arc_len = 2 * π * r * abs(angle) / 360
    n = trunc(arc_len / 4) + 3
    step_len = arc_len / n
    step_angle = angle / n

    # making a slight left turn before starting reduces
    # the error caused by the linear approximation of the arc
    
    t.left(step_angle/2)
    polyline(t, n, step_len, step_angle)
    t.right(step_angle/2)
    
end

arc

### Exercise 4-9

Write an appropriate set of functions (`petal`, and `flower`) that can draw turtle flowers.  `petal` will draw a single petal.  `flow` calls `petal` multiple times (like, $360/n$ times).

TIP: In `petal`, after the turtle finishes drawing the first arc, make it turn such that it is pointing the  correct direction to start the second arc.  When placed in a loop, these steps will make the turtle point the starting direction after it finishes drawing two arcs (that is, one petal).


In [21]:
#
function petal(t, r, angle)
  for i in 1:2
    arc(t, r, angle)
    t.left(180-angle)
  end
end 

function flower(t, n, r, angle)
  for i in 1:n
    petal(t, r, angle)
    t.left(360/n)
  end
end

flower (generic function with 1 method)

In [22]:
my_t.reset()
my_t.speed(1)

flower(my_t, 7, 100, 100)

### Exercise 4-10

Write an appropriately  set of functions that can draw shapes as in Turtle pies.  One suggestion is to define a function `isosceles` that draws an isosceles and a function that draws multiple isosceles next to one another function `polypie` to form a Turtle pie.  Optionally, a final function `drawpie` draws a polypie and then move the turtle outside the pie.

TIP: In writing a code for the first function, make sure you turn your turtle such that it is pointing the right direction after it finishes drawing an isosceles.

In [23]:
#
function isosceles(t, r, angle)
  y = r * sin(angle/2 * π / 180)
  t.right(angle/2)
  t.forward(r)
  t.left(90+angle/2)
  t.forward(2*y)
  t.left(90+angle/2)
  t.forward(r)
  t.left(180-angle/2)
end

function polypie(t, n, r)
  angle = 360 / n
  for i in 1:n
    isosceles(t, r, angle)
    t.left(angle)
  end
end

function drawpie(t, n, r)
  polypie(t, n, r)
  t.penup()
  t.forward(r*2 + 10)
  t.pendown()
end

drawpie (generic function with 1 method)

In [30]:
my_t.reset()
my_t.speed(5)
drawpie(my_t, 5, 100)

### Exercise 4-12

Read about spirals at https://en.wikipedia.org/wiki/Spiral; then write a program that draws an Archimedan spiral as in Archimedan spiral: 


${r = a + b{\theta}}$


Try something like

```
spiral(my_t, 750, 3, 0.1, 0.001)
```

In [28]:
function spiral(t, n, len, a, b)
  theta = 0.0
  for i in 1:n
    t.forward(len)
    dtheta = 1 / (a + b * theta)
    t.left(dtheta)
    theta += dtheta
   # println(dtheta,"  ", theta)
  end
end

spiral (generic function with 1 method)

In [33]:
my_t.reset()
my_t.speed(10)
spiral(my_t, 750, 3, 0.1, 0.001)